Installation. In Julia REPL this can be done easier using the packaging mode
(typing `]add COBREXA`)

In [1]:
import Pkg
Pkg.add("COBREXA")

using COBREXA

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Let's first get a simple model to have a look at

In [2]:
import Downloads
Downloads.download(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
)

"e_coli_core.json"

Load the model

In [3]:
ecoli = load_model("e_coli_core.json");

have a look at what the model contains. First, metabolites

In [4]:
metabolites(ecoli)

72-element Vector{String}:
 "glc__D_e"
 "gln__L_c"
 "gln__L_e"
 "glu__L_c"
 "glu__L_e"
 "glx_c"
 "h2o_c"
 "h2o_e"
 "h_c"
 "h_e"
 ⋮
 "f6p_c"
 "fdp_c"
 "for_c"
 "for_e"
 "fru_e"
 "fum_c"
 "fum_e"
 "g3p_c"
 "g6p_c"

reactions

In [5]:
reactions(ecoli)

95-element Vector{String}:
 "PFK"
 "PFL"
 "PGI"
 "PGK"
 "PGL"
 "ACALD"
 "AKGt2r"
 "PGM"
 "PIt2r"
 "ALCD2x"
 ⋮
 "MALt2_2"
 "MDH"
 "ME1"
 "ME2"
 "NADH16"
 "NADTRHD"
 "NH4t"
 "O2t"
 "PDH"

we can have a look at what each reaction does

In [6]:
reaction_stoichiometry(ecoli, "PFK")

Dict{String, Float64} with 5 entries:
  "adp_c" => 1.0
  "atp_c" => -1.0
  "f6p_c" => -1.0
  "fdp_c" => 1.0
  "h_c"   => 1.0

or look at which metabolites belong to which compartments (Julia looping!)

In [7]:
[m => metabolite_compartment(ecoli, m) for m in metabolites(ecoli)]

72-element Vector{Pair{String, String}}:
 "glc__D_e" => "e"
 "gln__L_c" => "c"
 "gln__L_e" => "e"
 "glu__L_c" => "c"
 "glu__L_e" => "e"
    "glx_c" => "c"
    "h2o_c" => "c"
    "h2o_e" => "e"
      "h_c" => "c"
      "h_e" => "e"
            ⋮
    "f6p_c" => "c"
    "fdp_c" => "c"
    "for_c" => "c"
    "for_e" => "e"
    "fru_e" => "e"
    "fum_c" => "c"
    "fum_e" => "e"
    "g3p_c" => "c"
    "g6p_c" => "c"

or try to get human-readable names from the reactions

In [8]:
sort([r => reaction_name(ecoli, r) for r in reactions(ecoli)])

95-element Vector{Pair{String, String}}:
    "ACALD" => "Acetaldehyde dehydrogenase (acetylating)"
   "ACALDt" => "Acetaldehyde reversible transport"
     "ACKr" => "Acetate kinase"
   "ACONTa" => "Aconitase (half-reaction A, Citrate hydro-lyase)"
   "ACONTb" => "Aconitase (half-reaction B, Isocitrate hydro-lyase)"
    "ACt2r" => "Acetate reversible transport via proton symport"
     "ADK1" => "Adenylate kinase"
    "AKGDH" => "2-Oxogluterate dehydrogenase"
   "AKGt2r" => "2 oxoglutarate reversible transport via symport"
   "ALCD2x" => "Alcohol dehydrogenase (ethanol)"
            ⋮
 "SUCCt2_2" => "Succinate transport via proton symport (2 H)"
   "SUCCt3" => "Succinate transport out via proton antiport"
    "SUCDi" => "Succinate dehydrogenase (irreversible)"
   "SUCOAS" => "Succinyl-CoA synthetase (ADP-forming)"
     "TALA" => "Transaldolase"
     "THD2" => "NAD(P) transhydrogenase"
     "TKT1" => "Transketolase"
     "TKT2" => "Transketolase"
      "TPI" => "Triose-phosphate isomerase

this is useful to e.g. quickly look up dehumanized gene IDs

In [9]:
gs = genes(ecoli)
sort(gene_name.(Ref(ecoli), gs) .=> gs)

137-element Vector{Pair{String, String}}:
     "" => "s0001"
 "aceA" => "b4015"
 "aceB" => "b4014"
 "aceE" => "b0114"
 "aceF" => "b0115"
 "ackA" => "b2296"
 "acnA" => "b1276"
 "acnB" => "b0118"
 "adhE" => "b1241"
 "adhP" => "b1478"
        ⋮
 "talB" => "b0008"
 "tdcD" => "b3115"
 "tdcE" => "b3114"
 "tktA" => "b2935"
 "tktB" => "b2465"
 "tpiA" => "b3919"
 "ydjI" => "b1773"
 "ytjC" => "b4395"
  "zwf" => "b1852"

the model's internals are described by a bipartite graph between reactions
and metabolites, commonly stoichiometry

In [10]:
stoichiometry(ecoli)

72×95 SparseArrays.SparseMatrixCSC{Float64, Int64} with 360 stored entries:
⎡⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢄⠀⠀⠀⠀⠀⠀⠀⠈⠶⠴⡆⠀⠀⠀⠀⠀⠀⎤
⎢⡀⢐⣀⢀⡀⡒⢒⣐⠀⣂⣂⠀⣂⣂⢂⠀⢀⠀⠀⠀⠀⠀⢀⠄⠀⠀⠀⢂⠀⢂⣀⣐⡒⡀⠆⢙⣀⠀⡀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⠈⢑⣀⣀⠀⠀⎥
⎢⠀⠀⠃⠀⠃⠀⠀⠀⠘⠀⡇⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⡜⠀⡄⣤⢠⠘⠙⢣⡇⠘⎥
⎢⠀⠐⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠐⠁⠉⠀⠀⠀⠀⠀⠘⠄⎥
⎢⠀⢐⠀⠂⠀⠄⠠⠠⠀⠠⠆⠀⠄⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠠⠀⠠⠀⠀⢀⠀⠀⠠⠀⠀⠁⎥
⎢⢀⠐⠀⠨⢀⠁⠈⣈⠀⢁⣁⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠁⢀⠀⢊⠉⠀⠀⠀⢀⠀⣀⠀⢀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡈⠀⡀⠆⠀⠆⠀⡀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠆⠀⎥
⎢⠀⠀⠂⠀⡂⠀⠀⠁⠀⠀⠀⠈⠁⠀⠀⠀⠄⠄⢁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⎥
⎢⠈⠀⠁⠀⠀⢀⡀⠀⠠⠁⠁⠀⠑⠀⠐⠲⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠀⠂⠀⠀⠀⠀⠀⠀⠊⠀⠀⠀⠈⎥
⎢⠄⠠⢠⠀⠰⠀⠠⠀⠤⠦⠄⠈⠀⠀⠀⠠⠀⠁⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⠄⠄⠠⠀⠀⠀⠀⠀⎥
⎢⠂⠐⠀⠀⠐⡠⢐⠘⢃⠒⠂⡀⠄⠀⠀⠐⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠒⠀⢀⢀⠀⠀⣀⠀⢀⎥
⎢⠈⠀⠁⠀⡀⠀⠀⠀⠈⠁⠅⠀⠁⠀⢀⠈⠄⠔⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠈⎥
⎢⠣⠁⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠈⠀⠁⠁⠀⠈⡀⠀⠀⠀⠀⠀⠐⢣⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⡄⠀⠀⠀⠀⠂⠄⠤⠀⠀⠈⠂⠀⠀⠀⠀⠠⠀⠊⠒⣠⠀⠀⠀⠀⠀⠀⠀⠀⠀⎦

(the matrix is likely zoomed out, but otherwise it's a normal matrix)

Let's try to build a tiny custom model
ref: https://lcsb-biocore.github.io/COBREXA.jl/stable/examples/04b_standardmodel_construction/
The model we will try to recreate looks like this small metabolic network:

![toy model](img/toy_model.png)

The model type for "manually constructed" models we call a `StandardModel`
because it is kinda standard way to handle stuff in cobra community.
Technically we can convert the above model to StandardModel and play with it:

In [11]:
ecoli = convert(StandardModel, ecoli)
ecoli.reactions["PFK"]
ecoli.reactions["PFK"].lb = 0.5

0.5

But we want to make a completely custom model

In [12]:
model = StandardModel("MyModel")

Metabolic model of type COBREXA.StandardModel
sparse(Int64[], Int64[], Float64[], 0, 0)
Number of reactions: 0
Number of metabolites: 0


let's make some metabolites

In [13]:
a = Metabolite("a", name = "molecule A", formula = "H2O", compartment = "outside")
b = Metabolite("b") #details omitted for demonstration
c = Metabolite("c")

Metabolite.id: c
Metabolite.name: ---
Metabolite.formula: ---
Metabolite.charge: ---
Metabolite.compartment: ---
Metabolite.notes: ---
Metabolite.annotations: ---


Push the prepared metabolites into the model (the ! stands for "execute!", it
is a syntactic convention for warning that the function changes some of the
parameters (the model) in place)

In [14]:
add_metabolites!(model, [a, b, c])

let's make some reactions

In [15]:
b1 = Reaction("b1", lb = 0.0, ub = 10.0);
b1.metabolites = Dict("a" => 1);

b2 = Reaction("b2", lb = 0.0, ub = 10.0);
b2.metabolites = Dict("b" => -1);

b3 = Reaction("b3", lb = 0.0, ub = 10.0);
b3.metabolites = Dict("c" => -1);

add the reactions to the model

In [16]:
add_reactions!(model, [b1, b2, b3])

shortcut for the above, makes an array of 3 reactions at once

In [17]:
list_of_reactions = [Reaction("v$i", lb = 0.0, ub = 10.0) for i = 1:4]

4-element Vector{COBREXA.Reaction}:
 COBREXA.Reaction("v1", nothing, Dict{String, Float64}(), 0.0, 10.0, nothing, nothing, Dict{String, Vector{String}}(), Dict{String, Vector{String}}(), 0.0)
 COBREXA.Reaction("v2", nothing, Dict{String, Float64}(), 0.0, 10.0, nothing, nothing, Dict{String, Vector{String}}(), Dict{String, Vector{String}}(), 0.0)
 COBREXA.Reaction("v3", nothing, Dict{String, Float64}(), 0.0, 10.0, nothing, nothing, Dict{String, Vector{String}}(), Dict{String, Vector{String}}(), 0.0)
 COBREXA.Reaction("v4", nothing, Dict{String, Float64}(), 0.0, 10.0, nothing, nothing, Dict{String, Vector{String}}(), Dict{String, Vector{String}}(), 0.0)

...and we can fill them in

In [18]:
list_of_reactions[1].metabolites = Dict("a" => -1, "b" => 1)
list_of_reactions[2].metabolites = Dict("a" => -1, "c" => 1)
list_of_reactions[3].metabolites = Dict("a" => 1, "c" => -1)
list_of_reactions[4].metabolites = Dict("b" => 1, "c" => -1)

Dict{String, Int64} with 2 entries:
  "c" => -1
  "b" => 1

We can have a look at individual reactions; this also formats them nicely as actual reactions.

In [19]:
list_of_reactions[1]

Reaction.id: v1
Reaction.name: ---
Reaction.metabolites: 1.0 a →  1.0 b
Reaction.lb: 0.0
Reaction.ub: 10.0
Reaction.grr: ---
Reaction.subsystem: ---
Reaction.notes: ---
Reaction.annotations: ---
Reaction.objective_coefficient: 0.0


add the reactions to the model

In [20]:
add_reactions!(model, list_of_reactions)

let's have a look at what it looks like as a matrix

In [21]:
stoichiometry(model)

3×7 SparseArrays.SparseMatrixCSC{Float64, Int64} with 11 stored entries:
 1.0    ⋅     ⋅   -1.0  -1.0   1.0    ⋅ 
  ⋅   -1.0    ⋅    1.0    ⋅     ⋅    1.0
  ⋅     ⋅   -1.0    ⋅    1.0  -1.0  -1.0

Now let's try to numerically solve the model. First, we need a linear solver;
GLPK will do a good job for this purpose. Other alternatives include Tulip
(native interior-point solver), OSQP and Clarabel (for quadratic problems),
SCIP (free and fast), Clp, and very good commercial solvers include Gurobi
and CPLEX (you will need to install the licenses for these manually).

In [22]:
Pkg.add("GLPK")
using GLPK

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


"Solving" the model is finding optimum with respect to some reaction, thus we
first need to choose the objective that the solver should actually optimize.
Here, let's maximize the flux through the reaction "v1".

In [23]:
change_objective!(model, "v1")

You can observe the result in the model reactions' objective coefficients
(and fine-tune that manually if needed):

In [24]:
model.reactions["v1"].objective_coefficient

1.0

Flux balance analysis finds a steady state flux through the model which is
within the reaction flux lower and upper bounds:

In [25]:
model_solution = flux_balance_analysis(model, GLPK.Optimizer)

A JuMP Model
Maximization problem with:
Variables: 7
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 14 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: lbs, mb, ubs, x

The above returned a solved optimization model description. From that we can
extract the actual value of the objective function:

In [26]:
solved_objective_value(model_solution)

10.0

...and the complete description of the flux through the reactions

In [27]:
flux_vector(model, model_solution)

7-element Vector{Float64}:
 10.0
 10.0
  0.0
 10.0
  0.0
  0.0
  0.0

...or a little better as a dictionary

In [28]:
solution_dict = flux_dict(model, model_solution)

Dict{String, Float64} with 7 entries:
  "v4" => 0.0
  "v2" => 0.0
  "b3" => 0.0
  "v1" => 10.0
  "b2" => 10.0
  "v3" => 0.0
  "b1" => 10.0

typically, one does all of this in one step with a shortcut function:

In [29]:
flux_balance_analysis_dict(model, GLPK.Optimizer)

Dict{String, Float64} with 7 entries:
  "v4" => 0.0
  "v2" => 0.0
  "b3" => 0.0
  "v1" => 10.0
  "b2" => 10.0
  "v3" => 0.0
  "b1" => 10.0

As a check, we can verify that the amount of metabolites in the model indeed
stays balanced; if the solver worked, this vector should be zero (or within
the numerical tolerance of zero):

In [30]:
stoichiometry(model) * flux_vector(model, model_solution)

3-element Vector{Float64}:
 0.0
 0.0
 0.0

Finally, it is very useful to be able to write the results to a file. In
Julia, we simply make a DataFrame (which behaves just as dataframes in other
languages) and use a CSV package to format it into a CSV file.

In [31]:
Pkg.add(["DataFrames", "CSV"])
using DataFrames, CSV
df = DataFrame(reaction = collect(keys(solution_dict)), flux = collect(values(solution_dict)))
CSV.write("toy_solution.csv", df)

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


"toy_solution.csv"

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*